In [1]:
"""
Function to load weight to spesific models
"""
def backbone_load(model,model_weight,keys=['input_conv','unet','offset_linear', 'output_layer', 'semantic_linear']):
    processed_dict = {}
    for k in net_weighth.keys(): 
        decomposed_key = k.split(".")[0]
        if(decomposed_key in keys):
            processed_dict[k] = net_weighth[k] 
    model_dict = model.state_dict()
    model_dict.update(model_weight)
    model.load_state_dict(model_dict)
    return model

In [2]:
import torch
import pandas
from model.softgroup import SoftGroup
import yaml
from munch import Munch
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [3]:
!echo $SPCONV_SAVED_WEIGHT_LAYOUT

In [4]:
torch.cuda.current_device()

0

In [5]:
# torch.load('tensors.pt', map_location=torch.device('cpu'))
data = torch.load("softgroup_stpls3d_spconv2.pth")
net_weighth= data["net"]

In [6]:
cfg_txt = open("configs/training_backbone/backbone_ASPP_DICELOSS.yaml", 'r').read()
cfg = Munch.fromDict(yaml.safe_load(cfg_txt))

In [7]:
model = SoftGroup(**cfg.model).cuda()

In [ ]:
# model = backbone_load(model,net_weighth)


In [11]:
model.is_DICELOSS

True

In [ ]:
from data import build_dataloader, build_dataset
from evaluation import (ScanNetEval, evaluate_offset_mae, evaluate_semantic_acc,
                                  evaluate_semantic_miou)
from model import SoftGroup
from util import (AverageMeter, SummaryWriter, build_optimizer, checkpoint_save,
                  collect_results_gpu, cosine_lr_after_step, get_dist_info,
                  get_max_memory, get_root_logger, init_dist, is_main_process,
                  is_multiple, is_power2, load_checkpoint)

In [ ]:
import os.path as osp
import os
import time
cfg.work_dir = osp.join('./work_dirs', osp.splitext(osp.basename("test"))[0])
os.makedirs(osp.abspath(cfg.work_dir), exist_ok=True)
timestamp = time.strftime('%Y%m%d_%H%M%S', time.localtime())
log_file = osp.join(cfg.work_dir, f'{timestamp}.log')
logger = get_root_logger(log_file=log_file)
logger.info(f'Config:\n{cfg_txt}')
# logger.info(f'Distributed: {args.dist}')
# logger.info(f'Mix precision training: {cfg.fp16}')
# shutil.copy(args.config, osp.join(cfg.work_dir, osp.basename(args.config)))
writer = SummaryWriter(cfg.work_dir)


In [ ]:
# train_set = build_dataset(cfg.data.train, logger)
# val_set = build_dataset(cfg.data.test, logger)
# train_loader = build_dataloader(
# train_set, training=True, dist=False, **cfg.dataloader.train)
# # for batch in train_loader:
# #     data_batch = batch
# #     break
# # torch.save(data_batch, 'dummy_data.pth')

In [8]:
data = torch.load("dummy_data.pth",map_location="cuda:0")
data_test = torch.load("dummy_data_test.pth",map_location="cuda:0")

In [11]:
data["coords_float"]

tensor([[ 30.5439,   9.3444,   0.8462],
        [ 30.9186,   9.7945,   1.0418],
        [ 31.2391,   9.8502,   1.0683],
        ...,
        [  3.5641, -26.4289,   5.5817],
        [  3.2307, -26.3940,   5.5394],
        [  3.8095, -26.3695,   6.3030]], device='cuda:0')

In [12]:
feat = data["coords_float"]

In [ ]:
# import spconv.pytorch as spconv
from lib.softgroup_ops import functions as softgroup_ops
import torch.nn.functional as F

In [ ]:
 # def forward_test(self, batch_idxs, voxel_coords, p2v_map, v2p_map, coords_float, feats,
 #                     semantic_labels, instance_labels, pt_offset_labels, spatial_shape, batch_size,
 #                     scan_ids, **kwargs):
color_feats = data_test["feats"]
if model.with_coords:
    feats = torch.cat((data["feats"], data_test["coords_float"]), 1)
voxel_feats = softgroup_ops.voxelization(data_test["feats"], data_test["p2v_map"])
input_buff = spconv.SparseConvTensor(voxel_feats, data_test["voxel_coords"].int(), data_test["spatial_shape"], data_test["batch_size"])
semantic_scores, pt_offsets, output_feats = model.forward_backbone(input_buff, data_test["v2p_map"])
semantic_preds = semantic_scores.max(1)[1]
ret = dict(
            scan_id=data_test["scan_ids"][0],
            coords_float=data_test["coords_float"].cpu().numpy(),
            color_feats=color_feats.cpu().numpy(),
            semantic_preds=semantic_preds.cpu().numpy(),
            semantic_labels=data_test["semantic_labels"].cpu().numpy(),
            offset_preds=pt_offsets.cpu().detach().numpy(),
            offset_labels=data_test["pt_offset_labels"].cpu().numpy(),
            instance_labels=data_test["instance_labels"].cpu().numpy())

proposals_idx, proposals_offset = model.forward_grouping(semantic_scores, pt_offsets,
                                                        data_test['batch_idxs'], data_test["coords_float"],
                                                        model.grouping_cfg)
inst_feats, inst_map = model.clusters_voxelization(proposals_idx, proposals_offset,
                                                  output_feats, data_test["coords_float"],
                                                  **model.instance_voxel_cfg)
_, cls_scores, iou_scores, mask_scores = model.forward_instance(inst_feats, inst_map)
# pred_instances = self.get_instances(scan_ids[0], proposals_idx, semantic_scores,
#                                     cls_scores, iou_scores, mask_scores)
# gt_instances = self.get_gt_instances(semantic_labels, instance_labels)
# ret.update(dict(pred_instances=pred_instances, gt_instances=gt_instances))


In [ ]:
# def get_instances(self, scan_id, proposals_idx, semantic_scores, cls_scores, iou_scores,
 #                   mask_scores):
num_instances = cls_scores.size(0)
num_points = semantic_scores.size(0)
cls_scores = cls_scores.softmax(1)
semantic_pred = semantic_scores.max(1)[1]
cls_pred_list, score_pred_list, mask_pred_list = [], [], []

In [ ]:
for i in range(model.instance_classes):
    cls_pred = cls_scores.new_full((num_instances, ), i + 1, dtype=torch.long)
    cur_cls_scores = cls_scores[:, i]
    cur_iou_scores = iou_scores[:, i]
    cur_mask_scores = mask_scores[:, i]
    score_pred = cur_cls_scores * cur_iou_scores.clamp(0, 1)
    mask_pred = torch.zeros((num_instances, num_points), dtype=torch.int, device='cuda')
    mask_inds = cur_mask_scores > model.test_cfg.mask_score_thr
    cur_proposals_idx = proposals_idx[mask_inds].long()
    mask_pred[cur_proposals_idx[:, 0], cur_proposals_idx[:, 1]] = 1

    # filter low score instance
    inds = cur_cls_scores > model.test_cfg.cls_score_thr
    cls_pred = cls_pred[inds]
    score_pred = score_pred[inds]
    mask_pred = mask_pred[inds]

    # filter too small instances
    npoint = mask_pred.sum(1)
    inds = npoint >= model.test_cfg.min_npoint
    cls_pred = cls_pred[inds]
    score_pred = score_pred[inds]
    mask_pred = mask_pred[inds]
    cls_pred_list.append(cls_pred)
    score_pred_list.append(score_pred)
    mask_pred_list.append(mask_pred)
    

# return instances

In [ ]:
cls_pred = torch.cat(cls_pred_list).cpu().numpy()
score_pred = torch.cat(score_pred_list).cpu().detach().numpy()
mask_pred = torch.cat(mask_pred_list).cpu().numpy()


In [ ]:
score_pred

In [ ]:
mask_pred.shape

In [ ]:

instances = []
for i in range(cls_pred.shape[0]):
    pred = {}
    pred['scan_id'] = scan_id
    pred['label_id'] = cls_pred[i]
    pred['conf'] = score_pred[i]
    # rle encode mask to save memory
    pred['pred_mask'] = rle_encode(mask_pred[i])
    instances.append(pred)

In [ ]:
score_pred.shape